In [37]:
import chess.pgn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import lichess.api
from lichess.format import PGN
import json

pgn = open("lichess_db_standard_rated_2016-05.pgn")
#print(len(pgn))
users = {}
#dfGames = pd.DataFrame(columns = ['Event', 'White', 'Black', 'Opening', 'WhiteElo', 'BlackElo'])
games_arr = []
for x in range(6000000):
    headers = chess.pgn.read_headers(pgn)
    opening = headers.get('Opening').split(':')[0]
    opening = opening.split(' #')[0]
    if headers.get('Event') == "Rated Classical game":
        row = [headers.get('Event'), headers.get('White'), headers.get('Black'), opening, headers.get('WhiteElo'), headers.get('BlackElo')]
        games_arr.append(row)
        #dfGames = dfGames.append(pd.Series(row, index = ['Event', 'White', 'Black', 'Opening', 'WhiteElo', 'BlackElo']), ignore_index = True)
        
        if users.get(headers.get('White')):
            users[headers.get('White')].append(opening)
        else:
            users[headers.get('White')] = [opening]
        
        if users.get(headers.get('Black')):
            users[headers.get('Black')].append(opening)
        else:
            users[headers.get('Black')] = [opening]
            
# dfGames.to_csv('out.csv', encoding='utf-8', index=False)
with open('users.json','w') as data: 
    json.dump(users, data)

In [23]:
from lichess.format import PYCHESS
basket_size_sum = 0
count = 0
for k in users:
    count += 1
    basket_size_sum += len(users[k])
    
print(basket_size_sum/count)

39.62776699798044


In [7]:
basketMap = {}
for user in users:
    try:
        pgns = lichess.api.user_games(user, max=20, format=PYCHESS)
    except:
        continue
    openings = set()
    for pgn in pgns:
        if pgn.headers.get('Event') == "Rated Classical game":
            #print(pgn.headers.get('Opening'))
            openings.add(pgn.headers.get('Opening'))
    basketMap[user] = list(openings)

In [3]:
import chess.pgn
pgn = open("lichess_db_standard_rated_2016-05.pgn")
for x in range(1):
    headers = chess.pgn.read_headers(pgn)
print(headers)
print(headers.get('White'))

Headers(Event='Rated Bullet tournament https://lichess.org/tournament/IaRkDsvp', Site='https://lichess.org/r0cYFhsy', White='GreatGig', Black='hackattack', Result='0-1', BlackElo='1809', BlackRatingDiff='+11', ECO='B01', Opening='Scandinavian Defense: Mieses-Kotroc Variation', Termination='Time forfeit', TimeControl='60+0', UTCDate='2016.04.30', UTCTime='22:00:03', WhiteElo='1777', WhiteRatingDiff='-11')
GreatGig


In [40]:
dfGames = pd.DataFrame(games_arr, columns = ['Event', 'White', 'Black', 'Opening', 'WhiteElo', 'BlackElo'])

In [41]:
print(len(games_arr))
dfGames.to_csv('out.csv', encoding='utf-8', index=False)

1501080


In [27]:
from itertools import islice

for item in islice(users, 1):
    print(users[item])

['Owen Defense', "King's Knight Opening: Normal Variation", 'Philidor Defense #3', 'Ruy Lopez: Steinitz Defense', 'Ruy Lopez: Steinitz Defense', 'Philidor Defense #3', 'Russian Game: Three Knights Game', "Van't Kruijs Opening", 'Italian Game', 'Scotch Game', 'Zukertort Opening', 'Alekhine Defense', 'Caro-Kann Defense', "Queen's Pawn Game: Symmetrical Variation", 'Three Knights Opening', 'Italian Game: Anti-Fried Liver Defense', 'Ruy Lopez: Steinitz Defense', "Bishop's Opening: Vienna Hybrid", "Italian Game: Two Knights Defense, Modern Bishop's Opening", 'Sicilian Defense: Bowdler Attack', 'Philidor Defense #3', 'Italian Game: Anti-Fried Liver Defense', 'Philidor Defense #3', "King's Pawn Game: Wayward Queen Attack, Kiddie Countergambit", "King's Pawn Game: McConnell Defense", 'Italian Game: Anti-Fried Liver Defense', "Van't Kruijs Opening", 'Sicilian Defense: Bowdler Attack', "Queen's Gambit Accepted", 'Scotch Game', 'French Defense: Exchange Variation', 'Philidor Defense #3', 'Italian

In [42]:
with open('users.json') as f:
    data = f.read()
    
idkdict = json.loads(data)
for item in islice(idkdict, 1):
    print(idkdict[item])

['Owen Defense', "King's Knight Opening", 'Philidor Defense', 'Ruy Lopez', 'Ruy Lopez', 'Philidor Defense', 'Russian Game', "Van't Kruijs Opening", 'Italian Game', 'Scotch Game', 'Zukertort Opening', 'Alekhine Defense', 'Caro-Kann Defense', "Queen's Pawn Game", 'Three Knights Opening', 'Italian Game', 'Ruy Lopez', "Bishop's Opening", 'Italian Game', 'Sicilian Defense', 'Philidor Defense', 'Italian Game', 'Philidor Defense', "King's Pawn Game", "King's Pawn Game", 'Italian Game', "Van't Kruijs Opening", 'Sicilian Defense', "Queen's Gambit Accepted", 'Scotch Game', 'French Defense', 'Philidor Defense', 'Italian Game', 'Ruy Lopez', 'French Defense', 'Vienna Game', 'Italian Game', 'Italian Game', 'Italian Game', 'French Defense', "Queen's Pawn Game", 'Sicilian Defense', "King's Pawn Game", "Queen's Gambit Accepted", 'Reti Opening', 'Four Knights Game', 'Scotch Game', "King's Pawn Game", 'Czech Defense', 'Sicilian Defense', 'Scotch Game', 'Ruy Lopez', 'Russian Game', "Queen's Gambit Accepte

In [43]:
df = pd.read_csv('out.csv')
print(df)

                        Event        White             Black  \
0        Rated Classical game    edumenoyo   platito_de_lefa   
1        Rated Classical game     a7md2010           dreypat   
2        Rated Classical game     raking41          mariofil   
3        Rated Classical game     JOHN14-6            naikus   
4        Rated Classical game      guevara          Sacchina   
...                       ...          ...               ...   
1501075  Rated Classical game  skailain279  pincheweylopezXD   
1501076  Rated Classical game     Zimerman         paramirez   
1501077  Rated Classical game       Airon8           ardbano   
1501078  Rated Classical game  Danieloaiza         dannyatac   
1501079  Rated Classical game  papadesarah         photomium   

                        Opening  WhiteElo  BlackElo  
0                  Owen Defense      1770      1625  
1             Hungarian Opening      1704      1764  
2                  Italian Game      1695      1648  
3              